In [1]:
import sys
import re
import numpy
import pandas

In [2]:
# Functions that are helpful in en-, decrypting

In [3]:
def get_words_length(text: str) -> list:
    length = []
    
    word_len = 0
    for s in text:
        if s == ' ':
            if word_len != 0:
                length.append(word_len)
            word_len = 0
            continue
        
        word_len = word_len + 1
    
    length.append(word_len)
    
    return length

km_reduce = lambda text: re.sub(' ', '', text)

def recover_spaces(text: str, length: list) -> str:
    i, recovered = 0, ''
    word_length = length.copy()
    
    for s in text:
        if i == word_length[0]:
            recovered = recovered + ' '
            i = 0
            del word_length[0]
            
            if len(word_length) == 0:
                break
    
        recovered = recovered + s
        i = i + 1
    
    return recovered

In [4]:
# Visener cipher coding

In [5]:
def transfer_to_codes(word: str, symbols: list) -> list:
    codes = []

    for letter in word:
        for i in range(len(symbols)):
            if letter == symbols[i]:
                codes.append(i) 

    return codes

In [6]:
transfer_to_symbols = lambda word, symbols: ''.join([symbols[x] for x in word])

In [7]:
def visener_encrypt(word: str, key: str, symbols: list) -> str:
    word, key = tuple([transfer_to_codes(x, symbols) for x in [word, key]])
    k, n = tuple(map(len, [key, symbols]))

    return transfer_to_symbols([(word[i] + key[i % k]) % n for i in range(len(word))], symbols)

In [8]:
def visener_decrypt(word: str, key: str, symbols: list) -> str:
    word, key = transfer_to_codes(word, symbols), transfer_to_codes(key, symbols)
    k, n = len(key), len(symbols)

    return transfer_to_symbols([(word[i] - key[i % k]) % n for i in range(len(word))], symbols)

In [9]:
# Some staff to make cryptanalisys of texts

In [10]:
def shift(word: str, s: int, symbols: list) -> str:
    word = transfer_to_codes(word, symbols)
    n = len(symbols)
    
    return transfer_to_symbols([(x + s) % n  for x in word], symbols)

In [11]:
def km_split(text: str, k: int) -> list:
    spl = []

    for i in range(k):
        spl.append(text[i::k])

    return spl

In [12]:
def km_count_freqs(text: str, symbols: list) -> list:
    freqs = [0 for x in symbols]

    for i in range(len(symbols)):
        for s in text:
            if symbols[i] == s:
                freqs[i] = freqs[i] + 1

    return freqs

In [13]:
def calc_index_of_coincidence(text: str, symbols: list):
    freqs = km_count_freqs(text, symbols)
    n = len(text)
    index = numpy.sum([x * (x - 1) for x in freqs]) / (n * (n - 1))
    
    return index

In [14]:
def apply_partitions(text: str, n: int, symbols: list):
    table = [[], []]
    for k in range(1, n + 1):
        table[0].append(k)
        index =numpy.average([calc_index_of_coincidence(x, symbols) for x in km_split(text, k)])
        table[1].append(index)
    
    return pandas.DataFrame(table[1], index=table[0], columns=['index_of_coincidence'])

In [15]:
def calc_mutual_index_of_coincidence(text1: str, text2: str, symbols: list):
    freqs1 = km_count_freqs(text1, symbols)
    freqs2 = km_count_freqs(text2, symbols)
    n, m = len(text1), len(text2)
    mi = numpy.sum([x * y for x, y in zip(freqs1, freqs2)]) / (n*m)
    
    return mi

In [16]:
def get_mutual_shift(text1: str, text2: str, symbols: list, *, show=True):
    table = [[], []]
    m = len(symbols)
    
    for k in range(1, m + 1):
        table[0].append(m - k)
        table[1].append(calc_mutual_index_of_coincidence(shift(text2, k, symbols), text1, symbols))
    
    frame = pandas.DataFrame(table[1], index=table[0], columns=['mutual_index_of_coincidence'])
    
    if show:
        return pandas.DataFrame(table[1], index=table[0], columns=['mutual_index_of_coincidence'])
    else:
        return frame.idxmax()[0]

In [17]:
# Cryptanalysis of some text uploaded from 'src.txt'

In [18]:
symbols = [chr(x) for x in range(32, 127) if x != ' ']
m = len(symbols)

In [19]:
f = open('src.txt')
text, key = f.read(), 'conception'
words_length =  get_words_length(text) 
    
reduced_text = km_reduce(text)
encrypted_text = visener_encrypt(reduced_text, key, symbols)
    
f.close()

In [20]:
print(encrypted_text[:250])

-Q\[XZiS^VX_hS[}lRQcQ_aI%H]KdTPcTGG_>]Qh#>^[/\cYgXRi^YXh^V\?Y^XWN^ZaYcLS^RZVbZd|&ec]Uf!^_\]e]JUci_^PXUbLGcZ>X^YWWEJcdZ_UTYc]QVZZY]K{HS[hdXvcPUPZK^Zw1cJYaWZ:lK^cITcSHVhS\TRd*&K]^OfTQU)Q_d]K]T=_d[UfaN^TZUaOT`lAXTR9WEJYdZUAEbTP_|V^\TEccSTTZKgTI[CSYVZc_d


In [21]:
frame0 = apply_partitions(encrypted_text, 15, symbols)
frame0.nlargest(5, 'index_of_coincidence')

,index_of_coincidence
10,0.052126
5,0.037155
15,0.034616
13,0.030218
11,0.029575


In [22]:
key_length = 10
lines = km_split(encrypted_text, key_length)

In [23]:
get_mutual_shift(lines[0], lines[1], symbols).nlargest(5, 'mutual_index_of_coincidence')

,mutual_index_of_coincidence
12,0.042899
13,0.034569
19,0.030820
8,0.030404
7,0.029988


In [24]:
shifts = [0,]

for i in range(1, key_length):
    shifts.append(get_mutual_shift(lines[0], lines[i], symbols, show=False))

In [25]:
shifts

[0, 12, 11, 1, 2, 13, 17, 6, 12, 11]

In [26]:
keys = []
for c in range(m):
    key_codes = []
    for s in shifts:
        key_codes.append((c + s) % m)
    
    keys.append(transfer_to_symbols(key_codes, symbols))

In [27]:
print(pandas.DataFrame(keys)[31:40])
print(pandas.DataFrame(keys)[61:71])

             0
31  ?KJ@ALPEKJ
32  @LKABMQFLK
33  AMLBCNRGML
34  BNMCDOSHNM
35  CONDEPTION
36  DPOEFQUJPO
37  EQPFGRVKQP
38  FRQGHSWLRQ
39  GSRHITXMSR
             0
61  ]ih^_jncih
62  ^ji_`kodji
63  _kj`alpekj
64  `lkabmqflk
65  amlbcnrgml
66  bnmcdoshnm
67  condeption
68  dpoefqujpo
69  eqpfgrvkqp
70  frqghswlrq


In [28]:
my_key = 'CONCEPTION'
recover_spaces(visener_decrypt(encrypted_text, my_key, symbols), words_length)

'i ". 83*5*/( 50 :06 M 8)"5 .03&_ w)"5 &-4& $"/ i 4":_ n08 i ,/08 */ :063 8*-- p6/*4) .& 8*5) $0/5&.15N b65 :06L 50 .: 6/\'0356/"5& 4)"3&t)06()" % 3010 \'1 *5:, &&1*/(Ly 068 0/G5- &"7&. &Na 5\' *345i 8 "/5&%5 0# &4 *-&/5[b &-*&7&. &Z. :4 )".&y 068 06-%/ &7&3, /08w )&/i ) "%) 01&r "3&-:L" 5- &"450 /$&" 8 &&,t 04 &&: 06* /0 637 *--"(&j 6455 0) &"3: 0638 03%4y 064 ":" 8 03%L" /%5 )&/a--5 )*/,L5 )*/,0 \'0 /&a /%% ":" /%/ *()56 /5*-" / &8. &&5*/(Nb65L5 )&:4 ":L: 06" 3&6 /40$*"#-&[i/5 )&8 *-%&3/&44L* /5 )&7 *--"(&L& 7&3:5)*/(* 4# 03*/(\' 03: 06La/%8 &N NN8 &% 0/ 054 )*/&8 *5)" /:5)*/(Le7&/5 )06(): 06" 3&8 &-$0.&N'

In [29]:
my_key = 'conception'
recover_spaces(visener_decrypt(encrypted_text, my_key, symbols), words_length)

"I am writing to you - what more? What else can I say? Now I know in your will Punish me with contempt. But you, to my unfortunate shareThougha d ropo fp ityk eeping,Y ouw on'tl eavem e.A tf irstI w antedt ob es ilent;B elievem e:m ys hameY ouw ouldn everk nowW henI h adh opeR arely,a tl easto ncea w eekT os eey oui no urv illageJ ustt oh eary ourw ordsY ous aya w ord,a ndt henAllt hink,t hinko fo neA ndd aya ndn ightu ntila n ewm eeting.But,t heys ay,y oua reu nsociable;Int hew ilderness,i nt hev illage,e verythingi sb oringf ory ou,Andw e. ..w ed on ots hinew itha nything,Event houghy oua rew elcome."